In [1]:
import pandas as pd
import numpy as np
import dash
import flask
from dash.dependencies import Input, Output
import dash_core_components as dcc
import dash_html_components as html
from pandas_datareader import data as web
from datetime import datetime as dt

In [2]:
import pymongo

In [3]:
from dotenv import load_dotenv
load_dotenv()
import os
password = os.getenv("PASSWORD")
username = os.getenv("USERNAME")

In [4]:
client = pymongo.MongoClient('mongodb://%s:%s@127.0.0.1' % (username, password))
db = client["1-grams"]

In [5]:
"""
import csv
wordlist = []
with open('wordlist.csv', 'rt') as f:
    reader = csv.reader(f)
    wordlist = list(reader)
"""

"\nimport csv\nwordlist = []\nwith open('wordlist.csv', 'rt') as f:\n    reader = csv.reader(f)\n    wordlist = list(reader)\n"

In [6]:
def load(word):
    try:
        df = pd.DataFrame(list(db["tweets"].find({ "word": word})))
        df = df.sort_values(by=['time'])
        df['year'] = [date.year for date in df['time']]
        df['day'] = [date.timetuple().tm_yday for date in df['time']]
        return df
    except:
        print(str("No timeseries found for "+word))
        df = pd.DataFrame()
        return df

# Input box for query

In [ ]:
external_stylesheets = ['styles.css']

server = flask.Flask(__name__)

app = dash.Dash(__name__, server=server, external_stylesheets=external_stylesheets)

app.layout = html.Div([
    html.H1('Twitter 1-grams'),
    dcc.Input(id='my-id', value='@realdonaldtrump', type='text'),
    dcc.Graph(id='my-graph'),
])

@app.callback(Output('my-graph', 'figure'), [Input('my-id', 'value')])
def update_graph(input_value):
    data=[]
    layout={
        'yaxis': {'autorange': 'reversed', 'type':'log'},
    }
    figure = {'data':data, 'layout':layout}
    values = input_value.split(',')
    for item in values:
        df = load(item)
        data.append({'x':df['time'], 'y':df['rank'],'name':item})
    return figure


if __name__ == '__main__':
    app.run_server(port=8051, host='0.0.0.0')

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://0.0.0.0:8051/ (Press CTRL+C to quit)
10.245.186.62 - - [18/Apr/2019 13:05:45] "GET / HTTP/1.1" 200 -
10.245.186.62 - - [18/Apr/2019 13:05:45] "GET /styles.css HTTP/1.1" 200 -
10.245.186.62 - - [18/Apr/2019 13:05:45] "GET /_dash-layout HTTP/1.1" 200 -
10.245.186.62 - - [18/Apr/2019 13:05:45] "GET /_favicon.ico HTTP/1.1" 200 -
10.245.186.62 - - [18/Apr/2019 13:05:45] "GET /_dash-dependencies HTTP/1.1" 200 -
10.245.186.62 - - [18/Apr/2019 13:05:46] "POST /_dash-update-component HTTP/1.1" 200 -


# Dropdown for query

In [ ]:
import numpy as np

In [ ]:
from numpy import random

In [ ]:
nums = random.randint(0,len(wordlist),size=10000)

In [ ]:
options = []
for num in nums:
    word = wordlist[num][0]
    options.append({'label': word,'value': word})

In [ ]:
external_stylesheets = ['styles.css']

server = flask.Flask(__name__)

app = dash.Dash(__name__, server=server, external_stylesheets=external_stylesheets)

app.layout = html.Div([
    html.H1('Twitter 1-grams'),
    html.Div([
        dcc.Dropdown(id='my-dropdown', options=options, value=['@realdonaldtrump','covfefe'], multi=True)]),
    dcc.Graph(id='my-graph')
])

@app.callback(Output('my-graph', 'figure'), [Input('my-dropdown', 'value')])

def update_graph(input_value):
    data=[]
    figure = {'data':data}
    for item in input_value:
        df = load(item)
        data.append({'x':df['time'], 'y':df['counts'],'name':item})
    return figure


if __name__ == '__main__':
    app.run_server()